In [ ]:

import argparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def main(dataset_file, metric):
    df = pd.read_csv(dataset_file)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_data = scaler.fit_transform(df[metric].values.reshape(-1, 1))

    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)

    time_steps = 10
    X, y = create_dataset(scaled_data, time_steps)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    loss = model.evaluate(X_test, y_test, verbose=0)
    print(f"Loss in the test data for {metric}:", loss)

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    mse = mean_squared_error(y_test, predictions)
    print(f"Mean square error in test data for {metric}:", mse)

    plt.figure(figsize=(14, 7))
    plt.plot(y_test, label='True')
    plt.plot(predictions, label='Predicted')
    plt.title(f"Predictions vs. true values for {metric}")
    plt.xlabel("Index")
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.show()

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Train and evaluate an LSTM model for prediction.')
    parser.add_argument('dataset_file', type=str, help='Path to the dataset file')
    parser.add_argument('metric', type=str, choices=['throughput', 'latency', 'congestion', 'jitter', 'packet_loss'], help='Metric to train the model for')
    args = parser.parse_args()

    main(args.dataset_file, args.metric)


In [ ]:
python predictionLSTM.py dataset.csv throughput
